#### Génération des URL de récupération des séquences et récupération par scrapping

In [2]:
import numpy as np
import pandas as pd

In [3]:
bdd = pd.read_csv("rise_human_transcriptome.txt", sep = "\t")

/home/kooli/anaconda2/envs/py36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (1,2,4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:

# bdd convertie panda -> numpy array et seuls les 70 000 premiers RRIs retenus
arrbdd = np.array(bdd)[:70000]
arrbdd

#enlever 'chr' et formater les numéros de chromosomes
    
for i,el in enumerate(arrbdd[:,3]):
    arrbdd[i,0] = arrbdd[i,0][3:]
    if arrbdd[i,0] == 'X':
        arrbdd[i,0] = '23'
    elif arrbdd[i,0] == 'Y':
        arrbdd[i,0] = '24'  
    elif(arrbdd[i,0] == 'M'):
        continue
    elif int(arrbdd[i,0])<10:
        arrbdd[i,0] = '0' + arrbdd[i,0]
   
for i,el in enumerate(arrbdd[:,3]):
    arrbdd[i,3] = arrbdd[i,3][3:]
    if arrbdd[i,3] == 'X':
        arrbdd[i,3] = '23'
    elif arrbdd[i,3] == 'Y':
        arrbdd[i,3] = '24'  
    elif(arrbdd[i,3] == 'M'):
        continue
    elif int(arrbdd[i,3])<10:
        arrbdd[i,3] = '0' + arrbdd[i,3]
        

#enlever les chromosomes M
arg_sansM = np.argwhere((arrbdd[:,0] != 'M') & (arrbdd[:,3] != 'M'))
varrbdd = arrbdd[arg_sansM[:,0]]

length_varrbdd = varrbdd.shape[0]


#changement strand
for i in range(length_varrbdd) :
    if varrbdd[i,8] == '+':
        varrbdd[i,8] = 'true'
    elif varrbdd[i,8] == '-':
        varrbdd[i,8] = 'false'
    if varrbdd[i,9] == '+':
        varrbdd[i,9] = 'true'
    elif varrbdd[i,9] == '-':
        varrbdd[i,9] = 'false'


for i in range(length_varrbdd):
    varrbdd[i,1] = str(varrbdd[i,1])
    varrbdd[i,2] = str(varrbdd[i,2])
    varrbdd[i,4] = str(varrbdd[i,4])
    varrbdd[i,5] = str(varrbdd[i,5])





#génération position début et fin de séquence (nucléotides)
for i in range(length_varrbdd):
    milieu = int((int(varrbdd[i,1])+int(varrbdd[i,2]))/2)
    varrbdd[i,1] = str(milieu-100)
    varrbdd[i,2] = str(milieu+100)
    milieu = int((int(varrbdd[i,4])+int(varrbdd[i,5]))/2)
    varrbdd[i,4] = str(milieu-100)
    varrbdd[i,5] = str(milieu+100)

#génération URLs
for i in range(length_varrbdd):
    varrbdd[i,6] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,0]+"?report=genbank&from="+varrbdd[i,1]+"&to="+varrbdd[i,2]+"&strand="+varrbdd[i,8]+".html"

for i,el in enumerate(varrbdd[:,7]):
    varrbdd[i,7] = "https://www.ncbi.nlm.nih.gov/nuccore/NC_0000"+varrbdd[i,3]+"?report=genbank&from="+varrbdd[i,4]+"&to="+varrbdd[i,5]+"&strand="+varrbdd[i,9]+".html"
        






In [5]:
from bs4 import BeautifulSoup
from selenium import webdriver
import time
browser = webdriver.Firefox()

In [6]:
#for i,el in enumerate(varrbdd[:,18]):
    #url = varrbdd[i,6]
    #browser.get(url)
    #time.sleep(5)
    #innerHTML = browser.execute_script("return document.body.innerHTML")
    #soup = BeautifulSoup(innerHTML,"lxml")
    #seq = soup.find_all("span",{'class':"ff_line"})
    #varrbdd[i,18] = seq[0].text+seq[1].text+seq[2].text+seq[3].text

In [7]:
url = varrbdd[100,7]
browser.get(url)
time.sleep(8)
innerHTML = browser.execute_script("return document.body.innerHTML")
soup = BeautifulSoup(innerHTML,"lxml")
seq = soup.find_all("span",{'class':"ff_line"})
#seq = soup.find_all("div",{'class':"seq gbff"}) doesn't work for me 
rep = ''
for i in range(len(seq)):
    rep += seq[i].text
rep

'aaagagcagg gacccgataa gaagccattt ttcaaatgga aggccgtact gattttctggaacactgccc tgggaggaac tgcctggact ggcctcccta ccttcccagg aagcagttttattcatagtg gaagactcca catttgacat agaaagtgca gtagggttat acacattccctgtgttcaaa gttaggttgg a'

In [8]:
def former_sequence(sequence):
    return ''.join(sequence.split());

In [9]:
def transformer_sequence(sequence,long):
    nucleotides = np.zeros((4,long))
    cnt = 0
    for lettre in sequence:
        if lettre=='a':
            nucleotides[0,cnt] = 1
            cnt += 1
        elif lettre=='c':
            nucleotides[1,cnt] = 1
            cnt += 1
        elif lettre=='g':
            nucleotides[2,cnt] = 1
            cnt += 1
        elif lettre=='t':
            nucleotides[3,cnt] = 1
            cnt += 1
    return nucleotides

In [10]:
s = transformer_sequence(rep,201)
s.shape
seq = former_sequence(rep)
seq

'aaagagcagggacccgataagaagccatttttcaaatggaaggccgtactgattttctggaacactgccctgggaggaactgcctggactggcctccctaccttcccaggaagcagttttattcatagtggaagactccacatttgacatagaaagtgcagtagggttatacacattccctgtgttcaaagttaggttgga'

#### A partir de mainenant, on essaye d'obtenir les probabilités d'accesibilité de chaque nucléotide sur une séquence quelconque ARN par exemple 'ACGUUGUCACACGAU'

In [11]:
import RNA

ModuleNotFoundError: No module named 'RNA'

In [11]:
help(RNA.pfl_fold)

Help on built-in function pfl_fold in module _RNA:

pfl_fold(...)



In [29]:
len('augcucgucgaucguaguagcucaguacgucaguacgaucgacuacgacuguacguacgacugacguaugcuagcugacucgaucguaucgaucgacugcau')

102

In [37]:
a = RNA.pfl_fold_up('augcucgucgaucguaguagcucaguacgucaguacgaucgacuacgacuguacguacgacugacguaugcuagcugacucgaucguaucgaucgacugcau',1,102,102)

In [56]:
b = np.array(a)
b = b[1:,1]
b

NameError: name 'a' is not defined